In [38]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import os
import scipy.misc
import scipy
from tensorflow.examples.tutorials.mnist import input_data

In [39]:
def save_images(images, size, image_path):
    return imsave(images, size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

In [40]:
def generator(z):
    
    zP = slim.fully_connected(z,4*4*256,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.tanh,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,256])
    
    gen1 = slim.convolution2d_transpose(zCon,num_outputs=64,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm, activation_fn=tf.nn.tanh,scope='g_conv1', weights_initializer=initializer)
    
    gen2 = slim.convolution2d_transpose(gen1,num_outputs=32,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm, activation_fn=tf.nn.tanh,scope='g_conv2', weights_initializer=initializer)
    
    gen3 = slim.convolution2d_transpose(gen2,num_outputs=16,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm, activation_fn=tf.nn.tanh,scope='g_conv3', weights_initializer=initializer)
    
    g_out = slim.convolution2d_transpose(gen3,num_outputs=1,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh, scope='g_out', weights_initializer=initializer)
    
    return g_out


def discriminator(bottom, reuse=False):
    
    dis1 = slim.convolution2d(bottom,16,[4,4],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.sigmoid, reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    
    dis2 = slim.convolution2d(dis1,32,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=tf.nn.sigmoid, reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    
    dis3 = slim.convolution2d(dis2,64,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=tf.nn.sigmoid, reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    
    d_out = slim.fully_connected(slim.flatten(dis3),1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    return d_out

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

tf.reset_default_graph()

z_size = 100

initializer = tf.truncated_normal_initializer(stddev=0.02)

z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32)
real_in = tf.placeholder(shape=[None,32,32,1],dtype=tf.float32)

Gz = generator(z_in)
Dx = discriminator(real_in)
Dg = discriminator(Gz,reuse=True)

d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg))
g_loss = -tf.reduce_mean(tf.log(Dg))

tvars = tf.trainable_variables()

trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
d_grads = trainerD.compute_gradients(d_loss,tvars[9:])
g_grads = trainerG.compute_gradients(g_loss,tvars[0:9])

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)

loss_gen = []
loss_dis = []

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
batch_size = 128
iterations = 20000
sample_directory = './figures_Arch2'
model_directory = './trainedModels_Arch2'

init = tf.initialize_all_variables()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for i in range(iterations):
        zs = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
        xs,_ = mnist.train.next_batch(batch_size)
        xs = (np.reshape(xs,[batch_size,28,28,1]) - 0.5) * 2.0
        xs = np.lib.pad(xs, ((0,0),(2,2),(2,2),(0,0)),'constant', constant_values=(-1, -1))
        _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs})
        _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs})
        _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs})
        if i % 10 == 0:
            loss_gen.append(float(gLoss))
            loss_dis.append(float(dLoss))
            z2 = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
            newZ = sess.run(Gz,feed_dict={z_in:z2})
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            save_images(np.reshape(newZ[0:36],[36,32,32]),[6,6],sample_directory+'/fig'+str(i)+'.png')
        if i % 1000 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
text_file.close()
text_file1.close()
print ("Done")

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
print (loss_gen)

In [ ]:
print (loss_dis)